# PLN Backward Chaining

Backward Chaining is (or planned to be) used in 
1. OpenCog for OpenPsi planning,
2. improving fitness estimates of MOSES models,
3. generally finding subtler patterns that specialized algorithms like the pattern miner miss.
For a Background on PLN see Probabilistic logic networks.

## Theory

-  In simple rule-based systems, there are two kinds of inference, forward chaining and backward chaining.

Backward chaining: given a target, an atom that may or not contain free variables, the system tries to find an inference tree that proves the target, grounding the free variables if necessary.
Because the target determines which rules are selected and used, this method is considered goal-driven, in contrast to data-driven forward-chaining inference. Backwards chaining is also considered to be goal-directed inference, or hypothesis driven, because inferences are not performed until the system is made to prove a particular goal (i.e. a question).
Backward Chaining is used when you want to work backwards from the Consequent to the Antecedent. It is usually the part that follows 'then' in a proposition.

#### Examples of antecedents & consequents:
-  If __P__ then __Q__.
__Q__ is the consequent of this hypothetical proposition.__P__ is the antecedent.
-  If __X__ is a mammal, then __X__ is an animal.
Here, " __X__ is an animal" is the consequent, and "__X__ is a mammal" an antecedent.
-  If OpenCog can think, then the Singularity is near.
"the Singularity is near" is the consequent, and "OpenCog can think" is an antecedent.

### A logical example of Backward Chaining
Here we will try to satisfy the goal of determining whether 'Plato is mortal' or not.
Known facts:
-  Plato is a philosopher
-  Plato likes to wrestle
The goal is to find out whether Plato is mortal based on a rule base:
1. If X is a philosopher and likes to wrestle then X is a man
2. If X can fly and lives in the sky then X is a god
3. If X is a god then X is immortal
4. If X is a man then X is mortal
Using backwards reasoning we can determine whether Plato is mortal in 4 steps. As we can see from rule 4, that this rule has a consequent of 'X is mortal' matches our goal to find out if 'Plato is mortal', so this checks out. Look at the antecedent to step 4 (the bit just after the If clause) - it is 'X is a man'. So now this antecedent (Substituting X for Plato) becomes the new goal of finding out if 'Plato is a man'. 
We can see rule 1 contains the consequent 'X is a man', and can see the antecedents are .. 'X is a philosopher', and 'X likes to wrestle'.
So now the antecedent(s) are broken down into two subgoals... 
Given the two facts of 'Plato is a Philosopher' and 'Plato likes to wrestle', we can see that these two subgoals are satisfied. 
We find rule 1 matches - and find that it is true that the consequent of rule 4 'X is mortal' matches - and therefore Plato is indeed mortal.

### More theory
Here is short explainer of Backward Chaining by Jocelyn Ireson-Paine
Also see the Wikipedia Page on Backward Chaining

### Practice
The following example is a simplified version of frog and relies on some of its files. So first go under

cd atomspace/examples/rule-engine/frog
and fire up guile by typing 

guile 

into a terminal then set the logging level to debug (will be handy to understand what the backward chainer does)

In [28]:
(use-modules (ice-9 readline)) 
(activate-readline)
(add-to-load-path "/usr/local/share/opencog/scm")
(add-to-load-path ".")
(use-modules (opencog))
(use-modules (opencog query))
(use-modules (opencog exec))
(use-modules (opencog logger))
(cog-logger-set-level! "debug")

"DEBUG"

In this example we have a black box for which we know there is something in it, and it makes croaking sounds and eats flies. The objective is to find the color of the thing in the black box. We have the following relations defined
-  If X croaks and X eats flies - Then X is a frog
-  If X is a frog - Then X is green
Let's say the thing in black box is named Fritz and from above relations we need to deduce its color.
Now load the rule-engine module, as well as the knowledge and rule bases

In [29]:
(use-modules (opencog rule-engine))

#<unspecified>

on guile, run 

(load "knowledge-base.scm")

(load "rule-base.scm")

Here are the file contents

In [30]:
;;;;;;;;;;;;;;;;;;;;
;; Knowledge base ;;
;;;;;;;;;;;;;;;;;;;;

(ImplicationScope (stv 1.0 1.0)
   (TypedVariable
      (Variable "$X")
      (Type "ConceptNode"))
   (And
      (Evaluation
         (Predicate "croaks")
         (Variable "$X"))
      (Evaluation
         (Predicate "eats_flies")
         (Variable "$X")))
   (Inheritance
      (Variable "$X")
      (Concept "frog")))

(ImplicationScope (stv 1.0 1.0)
   (TypedVariable
      (Variable "$X")
      (Type "ConceptNode"))
   (And
      (Evaluation
         (Predicate "chirps")
         (Variable "$X"))
      (Evaluation
         (Predicate "sings")
         (Variable "$X")))
   (Inheritance
      (Variable "$X")
      (Concept "Canary")))

(ImplicationScope (stv 1.0 1.0)
   (TypedVariable
      (Variable "$X")
      (Type "ConceptNode"))
   (Inheritance
      (Variable "$X")
      (Concept "frog"))
   (Inheritance
      (Variable "$X")
      (Concept "green")))

(ImplicationScope (stv 1.0 1.0)
   (TypedVariable
      (Variable "$X")
      (Type "ConceptNode"))
   (Inheritance
      (Variable "$X")
      (Concept "Canary"))
   (Inheritance
      (Variable "$X")
      (Concept "yellow")))

(Evaluation (stv 1.0 1.0)
   (Predicate "croaks")
   (Concept "Fritz"))

(Evaluation (stv 1.0 1.0)
   (Predicate "chirps")
   (Concept "Tweety"))

(Inheritance (stv 1.0 1.0)
   (Concept "Tweety")
   (Concept "yellow"))

(Evaluation (stv 1.0 1.0)
   (Predicate "eats_flies")
   (Concept "Tweety"))

(Evaluation (stv 1.0 1.0)
   (Predicate "eats_flies")
   (Concept "Fritz"))

(EvaluationLink (stv 1 1)
   (PredicateNode "eats_flies")
   (ConceptNode "Fritz")
)


The following two are files - conditional-instantiation-meta-rule.scm and fuzzy-conjunction-introduction-rule.scm which are required by rule-base.scm you may load them in guile as follows -

(load "../meta-rules/conditional-instantiation-meta-rule.scm")
(load "../rules/fuzzy-conjunction-introduction-rule.scm")

In [31]:
; =============================================================================
; Fuzzy conjunction introduction rule
;
; A1
; ...
; An
; |-
; AndLink
;   A1
;   ...
;   An
;
; Where A1 to An are atoms with a fuzzy TV
; -----------------------------------------------------------------------------

(use-modules (opencog rule-engine))
(use-modules (srfi srfi-1))

;; Generate variable (Variable prefix + "-" + to_string(i))
(define (gen-variable prefix i)
  (Variable (string-append prefix "-" (number->string i))))

;; Generate a list of variables (Variable prefix + "-" + to_string(n))
(define (gen-variables prefix n)
  (if (= n 0)
      ;; Base case
      '()
      ;; Recursive case
      (append (gen-variables prefix (- n 1))
              (list (gen-variable prefix (- n 1))))))

;; Generate a fuzzy conjunction introduction rule for an n-ary
;; conjunction
(define (gen-fuzzy-conjunction-introduction-rule nary)
  (let* ((variables (gen-variables "$X" nary))
         (EvaluationT (Type "EvaluationLink"))
         (InheritanceT (Type "InheritanceLink"))
         (type (TypeChoice EvaluationT InheritanceT))
         (gen-typed-variable (lambda (x) (TypedVariable x type)))
         (vardecl (VariableList (map gen-typed-variable variables)))
         (pattern (And variables))
         (rewrite (ExecutionOutput
                    (GroundedSchema "scm: fuzzy-conjunction-introduction-formula")
                    ;; We wrap the variables in Set because the order
                    ;; doesn't matter and that way alpha-conversion
                    ;; works better.
                    (List (And variables) (Set variables)))))
    (Bind
      vardecl
      pattern
      rewrite)))

;; Return true if all elements of the list are unique
(define (is-set l)
  (cond ((null? l) #t)
        ((member (car l) (cdr l)) #f)
        (else (is-set (cdr l)))))

;; Check that they all are different, and have positive confidences
(define (fuzzy-conjunction-introduction-precondition S)
  (bool->tv (is-confident-enough-set (cog-outgoing-set S))))

(define (is-confident-enough-set andees)
  (let* ((confident-enough (lambda (A) (> (cog-stv-confidence A) 0))))
    (and (is-set andees)
         (every confident-enough andees))))

(define (fuzzy-conjunction-introduction-formula A S)
  (let* ((andees (cog-outgoing-set S))
         (min-s-atom (min-element-by-key andees cog-stv-strength))
         (min-c-atom (min-element-by-key andees cog-stv-confidence))
         (min-s (cog-stv-strength min-s-atom))
         (min-c (cog-stv-confidence min-c-atom)))
    (if (is-confident-enough-set andees)       ; only introduce meaningful
                                               ; conjunction of unique andees
        (cog-set-tv! A (stv min-s min-c)))))

;; Name the rules
(define fuzzy-conjunction-introduction-2ary-rule
  (gen-fuzzy-conjunction-introduction-rule 2))
(define fuzzy-conjunction-introduction-2ary-rule-name
  (DefinedSchema "fuzzy-conjunction-introduction-2ary-rule"))

#<unspecified>

In [32]:
(DefineLink
  fuzzy-conjunction-introduction-2ary-rule-name
  fuzzy-conjunction-introduction-2ary-rule)

(DefineLink
   (DefinedSchemaNode "fuzzy-conjunction-introduction-2ary-rule")
   (BindLink
      (VariableList
         (TypedVariableLink
            (VariableNode "$X-0")
            (TypeChoice
               (TypeNode "EvaluationLink")
               (TypeNode "InheritanceLink")
            )
         )
         (TypedVariableLink
            (VariableNode "$X-1")
            (TypeChoice
               (TypeNode "EvaluationLink")
               (TypeNode "InheritanceLink")
            )
         )
      )
      (AndLink
         (VariableNode "$X-0")
         (VariableNode "$X-1")
      )
      (ExecutionOutputLink
         (GroundedSchemaNode "scm: fuzzy-conjunction-introduction-formula")
         (ListLink
            (AndLink
               (VariableNode "$X-0")
               (VariableNode "$X-1")
            )
            (SetLink
               (VariableNode "$X-0")
               (VariableNode "$X-1")
            )
         )
      )
   )
)


In [33]:
;;;;;;;;;;;;;;;;;;;;
;; Rule base ;;
;;;;;;;;;;;;;;;;;;;;

(define conditional-full-instantiation-meta-variables
  (VariableList
     (TypedVariable
        (Variable "$TyVs")
        (TypeChoice
           (Type "TypedVariableLink")
           (Type "VariableList")))
     (Variable "$P")
     (Variable "$Q")))

(define conditional-full-instantiation-meta-body
  (let* ((implication (Quote (ImplicationScope
                         (Unquote (Variable "$TyVs"))
                         (Unquote (Variable "$P"))
                         (Unquote (Variable "$Q")))))
         (precondition (Evaluation
                         (GroundedPredicate "scm: true-enough")
                         implication)))
  (And
    implication
    precondition)))

;; Here only the implicant is considered as premise. The variable(s)
;; should as well so the backward chainer can consider them as
;; targets, however that implies to lay them out explicitly, so for
;; sake of simplicity they are ignored for now.
(define conditional-full-instantiation-meta-rewrite
  (let* ((TyVs (Variable "$TyVs"))
         (P (Variable "$P"))
         (Q (Variable "$Q"))
         (implication (Quote (ImplicationScope
                         (Unquote TyVs)
                         (Unquote P)
                         (Unquote Q)))))
    (Quote (Bind
      (Unquote TyVs)
      (And
        (Unquote (LocalQuote (LocalQuote P)))
        (Evaluation (GroundedPredicate "scm: true-enough") (Unquote P)))
      (ExecutionOutput
        (GroundedSchema "scm: conditional-full-instantiation-formula")
        (Unquote
          (ListLink
            Q
            implication
            P)))))))

(define conditional-full-instantiation-meta-rule
  (BindLink
     conditional-full-instantiation-meta-variables
     conditional-full-instantiation-meta-body
     conditional-full-instantiation-meta-rewrite))

;; Return the TV of the evaluation of `an` assuming that it is an
;; AndLink, that is return the TV which is the the min over the
;; strengths and confidences of its outgoings. Normally this would be
;; handled by a rule but to simplify the unit test it is hardcored
;; here.
(define (conjunction-fuzzy-eval an)
  (let* ((outg (cog-outgoing-set an))
         (min-s-atom (min-element-by-key outg cog-stv-strength))
         (min-c-atom (min-element-by-key outg cog-stv-confidence))
         (min-s (cog-stv-strength min-s-atom))
         (min-c (cog-stv-confidence min-s-atom)))
    (stv min-s min-c)))

(define (true-enough-bool a)
  (let ((s (cog-stv-strength a)) (c (cog-stv-confidence a)))
    (and (> s 0.5) (> c 0.5))))

(define (true-enough a)
  (bool->tv (true-enough-bool a)))

;; Set (stv 1 1) on Q is Impl and P strength are both above 0.5 and
;; their confidence is non null.
(define (conditional-full-instantiation-formula Q Impl P)
  ;; Evaluate Q
  (if (and (true-enough-bool Impl) (true-enough-bool P))
      (cog-set-tv! Q (stv 1 1))))

;; Name the meta rule
(define conditional-full-instantiation-meta-rule-name
  (DefinedSchemaNode "conditional-full-instantiation-meta-rule"))

#<unspecified>

In [34]:
(DefineLink conditional-full-instantiation-meta-rule-name
  conditional-full-instantiation-meta-rule)

(DefineLink
   (DefinedSchemaNode "conditional-full-instantiation-meta-rule")
   (BindLink
      (VariableList
         (TypedVariableLink
            (VariableNode "$TyVs")
            (TypeChoice
               (TypeNode "TypedVariableLink")
               (TypeNode "VariableList")
            )
         )
         (VariableNode "$P")
         (VariableNode "$Q")
      )
      (AndLink
         (QuoteLink
            (ImplicationScopeLink
               (UnquoteLink
                  (VariableNode "$TyVs")
               )
               (UnquoteLink
                  (VariableNode "$P")
               )
               (UnquoteLink
                  (VariableNode "$Q")
               )
            )
         )
         (EvaluationLink
            (GroundedPredicateNode "scm: true-enough")
            (QuoteLink
               (ImplicationScopeLink
                  (UnquoteLink
                     (VariableNode "$TyVs")
                  )
                  (UnquoteLink
            

In [35]:
(define ci-rbs (ConceptNode "ci-rbs"))
(Inheritance ci-rbs (ConceptNode "URE"))

;; Associate the rules to the rule base (with weights, their semantics
;; is currently undefined, we might settled with probabilities but it's
;; not sure)
(MemberLink (stv 1 1)
   conditional-full-instantiation-meta-rule-name
   ci-rbs
)


(MemberLink (stv 1 1)
   (DefinedSchemaNode "conditional-full-instantiation-meta-rule")
   (ConceptNode "ci-rbs")
)


In [36]:
(MemberLink (stv 1 1)
   fuzzy-conjunction-introduction-2ary-rule-name
   ci-rbs
)

(MemberLink (stv 1 1)
   (DefinedSchemaNode "fuzzy-conjunction-introduction-2ary-rule")
   (ConceptNode "ci-rbs")
)


In [37]:
;; termination criteria parameters
(ExecutionLink
   (SchemaNode "URE:maximum-iterations")
   ci-rbs
   (NumberNode "20")
)


(ExecutionLink
   (SchemaNode "URE:maximum-iterations")
   (ConceptNode "ci-rbs")
   (NumberNode "20.000000")
)


In [38]:
;; Attention allocation (set the TV strength to 0 to disable it, 1 to
;; enable it)
(EvaluationLink (stv 0 1)
   (PredicateNode "URE:attention-allocation")
   ci-rbs
)

(EvaluationLink (stv 0 1)
   (PredicateNode "URE:attention-allocation")
   (ConceptNode "ci-rbs")
)


In knowledge-base.scm. You may observe ImplicationScopeLinks such as

(ImplicationScope (stv 1.0 1.0)

   (TypedVariable
   
      (Variable "$X")
      
      (Type "ConceptNode"))
      
   (And
      (Evaluation
      
         (Predicate "croaks")
         
         (Variable "$X"))
         
      (Evaluation
        (Predicate "eats_flies")
        
        (Variable "$X")))
        
   (Inheritance
   
      (Variable "$X")
      
      (Concept "frog")))
      
      
encoding the relationship

-  If X croaks and X eats flies - Then X is a frog
At the end of that same file, there are facts regarding Fritz and other animals as they relate to attributes using EvaluationLinks.
You may want to take a look at rule-base.scm which loads rules, associate them to a rule-base (called ci-rbs) and set some URE parameters (such as URE:maximum-iterations). As well as exploring the rule files conditional-instantiation-meta-rule.scm and fuzzy-conjunction-introduction-rule.scm as well. Don't worry if you don't understand it, just read the comments on the top of the files that explains what the rules are doing.
What the backward chainer is gonna do is use these rules over the knowledge base to find out what is green. We are going to call the backward chainer but let's first define the target

In [39]:
(define target
  (InheritanceLink (VariableNode "$what") (ConceptNode "green"))
)

#<unspecified>

meaning you want atoms that have an inheritance relationship with the concept green. To simplify the life the backward chainer and avoid non nonsensical answers (just as the target itself), let's restrict the type of the variable $what

In [40]:
(define vardecl
  (TypedVariable (VariableNode "$what") (TypeNode "ConceptNode"))
)

#<unspecified>

We may now call the backward chainer as follows

In [41]:
(cog-bc ci-rbs target #:vardecl vardecl)

(SetLink
   (InheritanceLink (stv 1 1)
      (ConceptNode "Fritz")
      (ConceptNode "green")
   )
)


$4 = (SetLink
   (InheritanceLink (stv 1 1)
      (ConceptNode "Fritz")
      (ConceptNode "green")
   )
)

If you wish to understand in more depth how the backward chainer came up with the answer, open the log file opencog.log under the same directory. Search the messages logged by the URE components (containing the string "[URE]") and try to make sense of what you see. :-) In particular you may find that the inference trees evolved by the backward chainer are logged right after the message "With inference tree:". For instance the inference tree corresponding to the solution is

                          [12543950459016998847][1] [15852562040974555944][1]
                          ======fuzzy-conjunction-introduction-formula=======
             [14398977004689637744][1] [15383395919935236078][1]
             ------conditional-full-instantiation-formula-------
[15603332628456784367][1] [13829542973489658864][1]
------conditional-full-instantiation-formula-------
             [13520875333812722983][1]
where the numbers corresponds the UUIDs of the atoms, target, intermediary targets and premises, involved with the inference. If you search these UUIDs you can get the corresponding atoms in the log file.

*If you dear reader think of some good quiz questions, add them to the discussion branch of this wiki page
Info

Maintained by Misgana, Nil (Practise)
Theory/Quiz added by Adam
Priority: Medium
add a good example (William Ma should pick the example)